<a href="https://colab.research.google.com/github/dongchanlim/CS450/blob/master/Week_11_Ensemble_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
file = files.upload()

Saving glass.csv to glass.csv


In [30]:
glass = pd.read_csv("glass.csv", skipinitialspace= True, na_values= ["?"])

In [31]:
glass.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [32]:
glass.dtypes

RI      float64
Na      float64
Mg      float64
Al      float64
Si      float64
K       float64
Ca      float64
Ba      float64
Fe      float64
Type      int64
dtype: object

In [34]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()

num_col = glass.drop(columns = "Type").select_dtypes(["number"]).columns

glass[num_col] = std.fit_transform(glass[num_col])

In [35]:
glass.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,0.872868,0.284953,1.254639,-0.692442,-1.127082,-0.671705,-0.145766,-0.352877,-0.586451,1
1,-0.249333,0.591817,0.636168,-0.170460,0.102319,-0.026213,-0.793734,-0.352877,-0.586451,1
2,-0.721318,0.149933,0.601422,0.190912,0.438787,-0.164533,-0.828949,-0.352877,-0.586451,1
3,-0.232831,-0.242853,0.698710,-0.310994,-0.052974,0.112107,-0.519052,-0.352877,-0.586451,1
4,-0.312045,-0.169205,0.650066,-0.411375,0.555256,0.081369,-0.624699,-0.352877,-0.586451,1


In [36]:
glass.shape

(214, 10)

In [37]:
glass.isnull().sum(axis = 0) / glass.shape[0] * 100

RI      0.0
Na      0.0
Mg      0.0
Al      0.0
Si      0.0
K       0.0
Ca      0.0
Ba      0.0
Fe      0.0
Type    0.0
dtype: float64

In [38]:
from sklearn.model_selection import train_test_split

X = glass.drop(columns = 'Type')
y = glass.Type.astype('category').cat.codes


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

1. Try at least 3 different "regular" learning algorithms and note the results.

In [39]:
# KNN classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors = np.int((np.sqrt(glass.size))), weights = "distance", n_jobs = -1)

knn.fit(X_train, y_train)

predicted_y = knn.predict(X_test)

accuracy_score(predicted_y, y_test)

0.6

In [40]:
# Naive Bayesian
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train, y_train)

predicted_y = nb.predict(X_test)

accuracy_score(predicted_y, y_test)

0.46153846153846156

In [44]:
# SVM
from sklearn.svm import SVC

svm = SVC(gamma='auto', kernel = 'rbf')

svm.fit(X_train, y_train)

predicted_y = svm.predict(X_test)

accuracy_score(predicted_y, y_test)

0.7076923076923077

2. Use a random forest and note the results. (Play around with a few different options)


In [45]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', max_depth = 4, n_jobs = -1, random_state = 0, class_weight = "balanced_subsample")

rfc.fit(X_train, y_train)

# predicted_y = rfc.predict(X_test)
# accuracy_score(predicted_y, y_test)

rfc.score(X_test, y_test)

0.6923076923076923

In [46]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 150, 200], 'criterion': ["gini",'entropy'], 'max_depth': [2,3,4,5], 'class_weight': ['balanced', 'balanced_subsample'], 'n_jobs' : [-1]}

search = GridSearchCV(rfc, param_grid, cv=5)

search.fit(X_train, y_train)

search.best_params_

{'class_weight': 'balanced_subsample',
 'criterion': 'entropy',
 'max_depth': 5,
 'n_estimators': 100,
 'n_jobs': -1}

In [47]:
search.best_score_

0.7519540229885058

In [48]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate= 0.5, max_depth= 4, random_state= 0)

gbc.fit(X_train, y_train)

gbc.score(X_test, y_test)

0.7076923076923077

In [49]:
param_grid = {'criterion': ['friedman_mse', 'mse', 'mae'], 'loss' :['deviance', 'exponential'], 'max_features':['auto', 'sqrt', 'log2'], 'max_depth': [2,3,4,5]}

search = GridSearchCV(gbc, param_grid, cv=5)

search.fit(X_train, y_train)

search.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: ExponentialLoss requires 2 classes; got 6 class(es)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: ExponentialLoss requires 2 classes; got 6 class(es)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: ExponentialLoss requires 2 classes; got 6 class(es)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estim

{'criterion': 'friedman_mse',
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': 'sqrt'}

In [50]:
search.best_score_

0.725287356321839